In [10]:
from math import comb
from scripts.models import PricingManager
from scripts.parse_pricing import parse_pricing
from itertools import chain, combinations

In [11]:
github = parse_pricing("pricings/2024/github.yml") # No dependencies between add-ons
petclinic = parse_pricing("pricings/extras/petclinic.yml") # Dependencies between add-ons

# Method 1

## Theory

### Definitions

- $C$: Configuration Space
- $P$: Set of Plans
- $a_p$: Set of add-ons available for plan $p$

### Mathematical Formulation

$$
|C| = |P| + \sum_{p \in P} \left( \sum_{k=1}^{|a_p|} \binom{|a_p|}{k} \right)
$$

### Consideration

This formulation assumes that all add-ons are independent of each other, meaning there are no dependencies that require the inclusion of other add-ons when selecting one, i.e. any combination of available add-ons for a plan is valid.

## Code

In [12]:
def calculate_subscriptions(pricing: PricingManager):
    
    np = len(pricing.plans)

    subscriptions = np

    add_ons_by_plan = {}

    for add_on in pricing.add_ons.values():
        for plan in add_on.available_for:
            add_ons_by_plan[plan] = add_ons_by_plan.get(plan, 0) + 1

    for plan, number_of_add_ons in add_ons_by_plan.items():
        for k in range(1, number_of_add_ons+1):
            subscriptions += comb(number_of_add_ons, k)
        
    return subscriptions

print(f"GitHub's pricing configuration space: {calculate_subscriptions(github)}")
print(f"PetClinic's pricing configuration space: {calculate_subscriptions(petclinic)}")

GitHub's pricing configuration space: 8960
PetClinic's pricing configuration space: 16


# Method 2

## Theory

### Definitions

- $C$: Configuration Space
- $P$: Set of Plans
- $A$: Set of add-ons
- $E(a)$: For each add-on \(a \in A\), \(E(a)\) is the set of plans for which a is excluded.
- $D(a)$: For each add-on \(a \in A\), \(D(a)\) is the set of add-ons on which \(a\) depends.

### Mathematical Formulation

**Compatible Add-ons for Each Plan**

For each plan $p \in P $, the set of add-ons ($A_p$) compatible with $p$ are:

$$
A_p = \{ a \in A \mid p \notin E(a) \}
$$

**Partial Order (Poset) Based on Dependencies**

So, for each plan $p$, que construct a partially ordered set (poset) ($(A_p, \leq)$), where the partial order $\leq$ is defined by:

$$
a_i \leq a_j \iff a_j \in D(a_i)
$$

This ensures that if an add-on $a_i$ depends on $a_j$, then $a_j$ must be included whenever $a_i$ is included.

**Ideals in the Poset**

An ideal in a poset ($A_p, \leq$) is a subset $I \leq A_p$ that is downward closed, meaning: 

If $a \in I$ and $b \leq a$, then $b \in I$.

The set of all ideals for plan $p$ is denoted as $I_p$:

$$
I_p = \{ \text{ideals in the poset } (A_p, \leq) \}
$$

​Each ideal represents a valid combination of add-ons that respects the dependencies.	

**Calculation of the size of the configuration space**

Finally, the total number of valid configurations $|C|$ is calculated as:

$$
|C| = |P| + \sum_{p \in P} |I_p|
$$

## Code

In [13]:
def powerset(iterable):
    "Generates all possible subsets (the power set) of an iterable."
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def is_valid_subset(subset, dependencies):
    "Checks if a subset meets the dependencies."
    subset_set = set(subset)
    for addon in subset_set:
        if not dependencies[addon].issubset(subset_set):
            return False
    return True

def calculate_ideals(compatible_addons, dependencies):
    "Calculates the number of ideals (valid configurations) in the poset of add-ons."
    count = 0
    for subset in powerset(compatible_addons):
        if is_valid_subset(subset, dependencies):
            count += 1
    return count

def calculate_subscriptions(pricing: PricingManager):
    total_subscriptions = 0

    for plan in pricing.plans:
        # Get the add-ons compatible with the plan
        compatible_addons = [name for name in pricing.add_ons.keys() if plan in pricing.add_ons[name].available_for]

        # Build the filtered dependencies for the compatible add-ons
        filtered_dependencies = {}
        for addon in compatible_addons:
            # Filter dependencies to include only compatible add-ons
            deps = pricing.add_ons[addon].depends_on or []
            filtered_dependencies[addon] = set(deps) & set(compatible_addons)

        # Calculate the number of ideals (valid configurations) in the poset
        num_ideals = calculate_ideals(compatible_addons, filtered_dependencies)

        # Add the number of valid configurations to the total
        total_subscriptions += num_ideals

    return total_subscriptions

print(f"GitHub's pricing configuration space: {calculate_subscriptions(github)}")
print(f"PetClinic's pricing configuration space: {calculate_subscriptions(petclinic)}")

GitHub's pricing configuration space: 8960
PetClinic's pricing configuration space: 14


# Conclusion

In short, for pricings that do not have exclusions between add-ons, the configuration space should be calculated using **Method 1** as it's much more faster. However, if there are dependencies between add-ons, **Method 2** should be used to ensure that the configuration space is calculated correctly.